# Explicit Finite Difference Scheme for Heston Model

## Explicit Scheme for Heston Model

### Parameters

In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
r = 0.05
kappa = 1.0
theta = 0.2
eta = 0.5
rho = -0.4
s0 = 100.0
v0 = 0.25
T = 1.0
K = 100.0

# Numerical solution parameters
smin = 0.0
smax = 300.0
vmin = 0.0
vmax = 2.0

### First price call option in Heston model, set up terminal condition

In [2]:
# Return a matrix as the terminal condition at time T
# Has to know ds and dv for the size of the matrix
def set_call_payoff(smin,smax,ds,vmin,vmax,dv,K):
    N = int(np.round((smax - smin) / ds))
    J = int(np.round((vmax - vmin) / dv))
    stk_price = np.linspace(smin,smax,N + 1)
    terminal_vec = np.maximum(stk_price - K,0)
    # Broadcast the vector to a matrix
    terminal_mat = np.tile(np.reshape(terminal_vec,(N + 1,1)),(1,J + 1))
    return terminal_mat

### Set up coefficients in explicit finite difference scheme

In [3]:
# Input n,j and output a_{n;j}
def set_a_FD(n,j,dt,dv,r,eta,theta,kappa,rho):
    return 1 - (dt / dv) * (r * dv + (n ** 2) * j * (dv ** 2) + (eta ** 2) * j)

# Input n,j and output b_{n;j}
def set_b_FD(n,j,dt,dv,r,eta,theta,kappa,rho):
    return (dt / 2) * ((n * r) + (n ** 2) * j * dv)
    
# Input n,j and output c_{n;j}
def set_c_FD(n,j,dt,dv,r,eta,theta,kappa,rho):
    return (dt / 2) * (-(n * r) + (n ** 2) * j * dv)

# Input n,j and output d_{n;j}
def set_d_FD(n,j,dt,dv,r,eta,theta,kappa,rho):
    return (dt / (2 * dv)) * ( kappa * (theta - j * dv) + (eta ** 2) * j)

# Input n,j and output e_{n;j}
def set_e_FD(n,j,dt,dv,r,eta,theta,kappa,rho):
    return (dt / (2 * dv)) * (-kappa * (theta - j * dv) + (eta ** 2) * j)

# Input n,j and output f_{n;j}
def set_f_FD(n,j,dt,dv,r,eta,theta,kappa,rho):
    return (dt * eta * j * n * rho) / 4

### Implement explicit finite difference

In [4]:
def explicit_FD_Heston(dt,ds,dv,T,smin,smax,vmin,vmax,r,kappa,theta,eta,rho,K):
    # Get size of matrix
    tmin = 0
    tmax = T
    M = int(np.round((tmax - tmin) / dt))
    N = int(np.round((smax - smin) / ds))
    J = int(np.round((vmax - vmin) / dv))
    
    # Only maintain two matrices of size (N + 1) * (J + 1) to record the 
    # numerical solution at all space variables
    # at the current time and the last time
    last_sol = set_call_payoff(smin,smax,ds,vmin,vmax,dv,K)
    sol = last_sol.copy()
    
    for m in range(M,0,-1):
        # Time index is m so derive C^{m-1} based on C^m
        # We have to derive C^{m-1}_{n;j} for all possible (n,j)
        
        # The first thing to do is to record the solution at the last time in last_sol
        # and we only update sol
        last_sol = sol
        for n in range(0,N + 1):
            # Stock price index
            for j in range(0,J + 1):
                # Volatility index
                
                # Check whether it hits boundary
                if n == 0:
                    sol[n,j] = 0
                    continue
                if n == N:
                    sol[n,j] = sol[n - 1,j] + ds
                    continue
                if j == 0:
                    sol[n,j] = (1 - r * dt) * last_sol[n,0]
                    continue
                if j == J:
                    sol[n,j] = n * ds
                    continue
                
                # Otherwise, get all coefficients
                a_FD = set_a_FD(n,j,dt,dv,r,eta,theta,kappa,rho)
                b_FD = set_b_FD(n,j,dt,dv,r,eta,theta,kappa,rho)
                c_FD = set_c_FD(n,j,dt,dv,r,eta,theta,kappa,rho)
                d_FD = set_d_FD(n,j,dt,dv,r,eta,theta,kappa,rho)
                e_FD = set_e_FD(n,j,dt,dv,r,eta,theta,kappa,rho)
                f_FD = set_f_FD(n,j,dt,dv,r,eta,theta,kappa,rho)
                
                # Perform update
                sol[n,j] = a_FD * last_sol[n,j] + b_FD * last_sol[n + 1,j] + \
                c_FD * last_sol[n - 1,j]\
                + d_FD * last_sol[n,j + 1] + e_FD * last_sol[n,j - 1]\
                + f_FD * (last_sol[n + 1,j + 1] + last_sol[n - 1,j - 1] - \
                          last_sol[n + 1,j - 1] - last_sol[n - 1,j + 1])
    return sol

### Run the function for different $\Delta t,\Delta s,\Delta v$

In [5]:
for dt in [0.00005]:
    for ds in [2.5]:
        for dv in [0.1]:
            # Do explicit finite difference scheme
            sol = explicit_FD_Heston(dt,ds,dv,T,smin,smax,vmin,vmax,
                                     r,kappa,theta,eta,rho,K)
            
            # Now take out the option price at s0 and v0
            s_ind = int(np.round((s0 - smin) / ds))
            v_ind = int(np.round((v0 - vmin) / dv))
            call_price = sol[s_ind,v_ind]
            
            # Put call parity
            put_price = call_price - s0 + K * np.exp(-r * T)
            print('Call price is: ' + str(call_price))
            print('Put price is: ' + str(put_price))

Call price is: 18.502326416954418
Put price is: 13.625268867025824


In [6]:
# The true call price in the Heston model is provided on the website  
# https://kluge.in-chemnitz.de/tools/pricer/
true_call = 20.4844

# Put call parity
true_put = true_call - s0 + K * np.exp(-r * T)
print('The true put price is: ' + str(true_put))

The true put price is: 15.6073424500714


## Try the explicit scheme for multiple values of $\Delta t,\Delta s$

In [7]:
# Record numerical solutions for fixed dt and ds
put_rec_mat = np.zeros((2,3))

for count_t,dt in enumerate([0.0001,0.00005]):
    for count_s,ds in enumerate([5,2.5,1.25]):
        for dv in [0.2]:
        # Fix the value of dv
            # Do explicit finite difference scheme
            sol = explicit_FD_Heston(dt,ds,dv,T,smin,smax,vmin,vmax,
                                     r,kappa,theta,eta,rho,K)
            
            # Now take out the option price at s0 and v0
            s_ind = int(np.round((s0 - smin) / ds))
            v_ind = int(np.round((v0 - vmin) / dv))
            call_price = sol[s_ind,v_ind]
            
            # Put call parity
            put_price = call_price - s0 + K * np.exp(-r * T)
            
            # Record the approximation
            put_rec_mat[count_t,count_s] = put_price
            
            # Output
            print('When dt = ' + str(dt) + ', ds = ' + str(ds) + \
                  ', the put price is ' + str(put_price))

When dt = 0.0001, ds = 5, the put price is 12.213552025490628


/var/folders/sd/ntm8vytx57g033nh4jm4j2700000gn/T/ipykernel_17065/388961646.py:50: RuntimeWarning: overflow encountered in double_scalars
  sol[n,j] = a_FD * last_sol[n,j] + b_FD * last_sol[n + 1,j] + \
/var/folders/sd/ntm8vytx57g033nh4jm4j2700000gn/T/ipykernel_17065/388961646.py:51: RuntimeWarning: overflow encountered in double_scalars
  c_FD * last_sol[n - 1,j]\
/var/folders/sd/ntm8vytx57g033nh4jm4j2700000gn/T/ipykernel_17065/388961646.py:50: RuntimeWarning: invalid value encountered in double_scalars
  sol[n,j] = a_FD * last_sol[n,j] + b_FD * last_sol[n + 1,j] + \


When dt = 0.0001, ds = 2.5, the put price is nan


/var/folders/sd/ntm8vytx57g033nh4jm4j2700000gn/T/ipykernel_17065/388961646.py:53: RuntimeWarning: invalid value encountered in double_scalars
  + f_FD * (last_sol[n + 1,j + 1] + last_sol[n - 1,j - 1] - \


When dt = 0.0001, ds = 1.25, the put price is nan
When dt = 5e-05, ds = 5, the put price is 12.195031464494235
When dt = 5e-05, ds = 2.5, the put price is 12.276560522834941
When dt = 5e-05, ds = 1.25, the put price is nan
